In [1]:
import csv
import os
import glob
import enchant
import pandas as pd
import itertools as it
from nltk.tokenize import sent_tokenize, word_tokenize,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


In [2]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv')

for i in range(df.index.max()+1):
    file=open('test/doc_'+str(i+1)+".tsv",'w',newline='')
    for j in range(9):
        file.write('%s\t'% df.iloc[i,j])
    file.close()

In [ ]:
"""
df = pd.read_csv('Airbnb_Texas_Rentals.csv')
df = df[df.drop('Unnamed: 0', axis= 1)]
"""

In [43]:

#Read the file and create a single tsv file for each record .  
with open('Airbnb_Texas_Rentals.csv') as csvin:
    reader = csv.reader(csvin)
    counter = 1
    for row in reader:
        with open("data/doc_"+str(counter)+".tsv",'w') as tsvout:
            writer = csv.writer(tsvout, delimiter='\t')
            writer.writerow(row)
            counter +=1

In [3]:
#Normalize the word. i.e. tokenize,stemming, and removing punctuations
def wordNorm(text):
    stemmer = SnowballStemmer("english")
    Normalized = []
    tokenizer = RegexpTokenizer(r'\w+')
    word_list = tokenizer.tokenize(text.lower())
    Remove_stopwords = [word for word in word_list if word not in stopwords.words('english')]
    stem_words = [stemmer.stem(word) for word in Remove_stopwords]
    
    d = enchant.Dict("en_US")
    for word in stem_words:
        if not word.isdigit() and len(word)>1 and d.check(word):#remove the punctuation
            Normalized.append(word)
    return Normalized

In [4]:
def readall_documents(path):
    words = []
    for file in glob.glob(path):
        df = pd.read_table(file,sep='\t',names=['average_rate_per_night','bedrooms_count',
                                                'city','date_of_listing','description','latitude',
                                                'longitude','title','url'])
        desc = wordNorm(str(list(df['description'])))
        title = wordNorm(str(list(df['title'])))
        for des, tit in it.zip_longest(desc,title):
            if(des is not None):
                words.append(des)
            if(tit is not None):
                words.append(tit)

    return words

In [5]:
#Read all the files inside test folder. Test folder contains all the tsv files
Allword = readall_documents('test/*')

In [7]:
def CreateVocabulary(allwords):
    Vocabulary = {}
    wordList = sorted(list(set(allwords)))
    for ID, element in enumerate(wordList):
        Vocabulary.update({element : ID})
    return Vocabulary

In [8]:
voc = CreateVocabulary(Allword)

In [10]:
def term_freq(term, text):
    count = 0
    if len(text) <= 0:
        return 0
    else:
        for ter in text:
            if ter == term:
                count += 1
        return count / len(text)

In [455]:
#Build invertedIndex
def invertedIndex(vocabulary,path):
    # invertedIndex = {termID : (doc, TF)}
    invertedIndex = {}
    counter = 0
    for file in glob.glob(path):
        df = pd.read_table(file,sep='\t',names=['index','average_rate_per_night','bedrooms_count',
                                                'city','date_of_listing','description','latitude',
                                                'longitude','title','url'])
        
        desc = wordNorm(str(list(df['description'])))
        title = wordNorm(str(list(df['title'])))
        for word in vocabulary:
            tf = term_freq(word, desc+title)
            if(tf>0):
                if(vocabulary[word] in invertedIndex.keys()):
                    invertedIndex[vocabulary[word]].append(file)
                else:
                    invertedIndex[vocabulary[word]] = [file]

    return invertedIndex

In [456]:
invIndex = invertedIndex(voc,'test/*')

In [481]:
query = "lovely land" #Accept the input

In [482]:
qnorm = wordNorm(query) #Normalize the query

In [483]:
#Find a document that contains all the words in the qnorm
inter = set()
for word in qnorm:
    if word in voc:
        if(len(inter) >0):
            inter = inter.intersection(set(invIndex[voc[word]]))
        else:
            inter = inter.union(invIndex[voc[word]])

In [484]:
for file in inter:
    df = pd.read_table(file,sep='\t',names=['index','average_rate_per_night','bedrooms_count',
                                                'city','date_of_listing','description','latitude',
                                                'longitude','title','url'])
    print("Title\n=========:")
    print(list(df['title']))
    print("\n Description\n=========:")
    print(list(df['description']))
    print("\n City\n=========:")
    print(list(df['city']))
    print("\n URL\n=========:")
    print(list(df['url']))

Title
=========:
['Texas Starlight Ranch']

 Description
=========:
['Texas Starlight Ranch sits on 18 acres of land and is also a nature preserve. My place is close to restaurants and dining and family-friendly activities. You’ll love my place because of the ambiance and the outdoors space. My place is good for couples, solo adventurers, business travelers, families (with kids), and big groups. Texas Starlight Ranch is also great for Holiday Parties for Thanksgiving, Christmas and New Years.']

 City
=========:
['McKinney']

 URL
=========:
[nan]
